In [1]:
#importing all required packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
user_details = pd.read_csv("takehome_users.csv",encoding='latin-1')
user_details.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [18]:
user_sessions = pd.read_csv("takehome_user_engagement.csv",encoding='latin-1')
user_sessions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
time_stamp    207917 non-null object
user_id       207917 non-null int64
visited       207917 non-null int64
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [5]:
user_details_req = user_details[['object_id','name','email','last_session_creation_time','org_id']]
user_details_req.head()

,object_id,name,email,last_session_creation_time,org_id
0,1,Clausen August,AugustCClausen@yahoo.com,1.398139e+09,11
1,2,Poole Matthew,MatthewPoole@gustr.com,1.396238e+09,1
2,3,Bottrill Mitchell,MitchellBottrill@gustr.com,1.363735e+09,94
3,4,Clausen Nicklas,NicklasSClausen@yahoo.com,1.369210e+09,1
4,5,Raw Grace,GraceRaw@yahoo.com,1.358850e+09,193


In [7]:
user_details_req['last_session_creation_time'] = pd.to_datetime(user_details_req['last_session_creation_time'],unit='s')

C:\Users\Lasya\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
user_details_req.head()

,object_id,name,email,last_session_creation_time,org_id
0,1,Clausen August,AugustCClausen@yahoo.com,2014-04-22 03:53:30,11
1,2,Poole Matthew,MatthewPoole@gustr.com,2014-03-31 03:45:04,1
2,3,Bottrill Mitchell,MitchellBottrill@gustr.com,2013-03-19 23:14:52,94
3,4,Clausen Nicklas,NicklasSClausen@yahoo.com,2013-05-22 08:09:28,1
4,5,Raw Grace,GraceRaw@yahoo.com,2013-01-22 10:14:20,193


In [25]:
data = user_sessions.groupby("user_id").filter(lambda x: len(x) > 3)

In [26]:
data.head(20)

,time_stamp,user_id,visited
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1
5,2013-12-31 03:45:04,2,1
6,2014-01-08 03:45:04,2,1
7,2014-02-03 03:45:04,2,1
8,2014-02-08 03:45:04,2,1
9,2014-02-09 03:45:04,2,1
10,2014-02-13 03:45:04,2,1


In [28]:
# first get all user ids
users = set(user_sessions['user_id'])
adopted_user = []

# iterate all users & check whether they are 'adopted user' using panda's API 'rolling'
for i, user in enumerate(users):
    df_user = user_sessions[user_sessions.user_id == user]
    df_user = df_user[['time_stamp', 'user_id']]
    df_user['time_stamp'] = pd.to_datetime(df_user['time_stamp'])
    df_user = df_user.set_index('time_stamp')
    df_user['at_least_3_days_over_7_days_period'] = df_user.rolling(window = '7D').count()
    
    if max(df_user['at_least_3_days_over_7_days_period']) >= 3:
        adopted_user.append(user)
        
print(len(adopted_user) / len(users) * 100)

18.157089425365523


About 18% of users are considered adopted users.